### Lemmatisierer Evaluierung - qualitativ

In [40]:
import glob
import json
import numpy as np
import os
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline

In [21]:
corpora = [l.split('lemmata\\')[1].strip('\\') for l in glob.glob("lemmata/*/") if not 'overview' in l]
algos = [l.split('lemmata/ud-gsd')[1].split('-ud-gsd.csv')[0].strip('\\') for l in glob.glob("lemmata/ud-gsd/*.csv")]
print(algos)
data = []
df_all = pd.DataFrame(columns=(['corpus', 'token', 'tag', 'tag_STTS', 'lemma']+algos[1:]))

['baseline', 'germalemma', 'gpt3', 'hanta-pretagged', 'hanta', 'rnntagger', 'simplemma', 'smorlemma', 'spacy2', 'spacy3', 'spacy33+', 'stanza-pretagged', 'stanza', 'trankit', 'treetagger']


In [86]:
# gpt3 not lemmatized all sentences
gpt3_all = pd.DataFrame(columns=(['corpus', 'token', 'tag', 'tag_STTS', 'lemma', 'gpt3']))
                        
for c in corpora:
    print(c)
    gold = pd.read_csv(f"lemmata/{c}/baseline-{c}.csv", encoding="utf-8", encoding_errors='ignore')
    if os.path.isfile(f"lemmata/{c}/gpt3-{c}.csv"):
        gpt =  pd.read_csv(f"lemmata/{c}/gpt3-{c}.csv", encoding="utf-8", encoding_errors='ignore')
        j = 0  # index in gpt3 file
        for i, line in gold.iterrows():
            if j < len(gpt):
                if list(line[:4]) == list(gpt.iloc[j, :4]):
                    gpt3_all.loc[len(gpt3_all)] = [c] + list(gpt.iloc[j, :5])
                    #print([c] + list(gpt.iloc[j, :4]))
                    j += 1
                else:
                    gpt3_all.append([c] + list(gpt.iloc[j, :4]) + [''])
                    j +=1
        #joined = gold.join(gpt, how='left', rsuffix='g')
    else:
        for i, line in gold.iterrows():
             gpt3_all.loc[len(gpt3_all)] = [c] +  list(line[:4]) + ['']

empirist-cmc-blog
empirist-cmc-blog-norm
empirist-cmc-chat-prof
empirist-cmc-chat-prof-norm
empirist-cmc-chat-social
empirist-cmc-chat-social-norm
empirist-cmc-twitter
empirist-cmc-twitter-norm
empirist-cmc-whatsapp
empirist-cmc-whatsapp-norm
empirist-cmc-wiki
empirist-cmc-wiki-norm
empirist-web
empirist-web-norm
germanc
germanc-norm
nosta-d-anselm-norm
nosta-d-anselm-orig
nosta-d-bematac-norm
nosta-d-bematac-orig
nosta-d-falko-norm
nosta-d-falko-orig
nosta-d-kafka-norm
nosta-d-kafka-orig
nosta-d-tuebadz-orig
nosta-d-unicum-norm
nosta-d-unicum-orig
rub2019-novelette
rub2019-opensubtitles
rub2019-sermononline
rub2019-ted
rub2019-wikipedia
tgermacorp
ud-gsd
ud-hdt
ud-pud


In [87]:
gpt3_all.to_csv("lemmata/gpt3_all.csv", encoding="utf-8")

In [88]:
for corpus in corpora:
    # prepare dataframe
    df = pd.read_csv(f"lemmata/{corpus}/baseline-{corpus}.csv", encoding="utf-8")#encoding_errors='ignore'
    df['corpus'] = corpus
    df.rename(columns = {'lemma_gold': 'lemma'}, inplace = True)
    df = df.drop(['lemma_pred'], axis=1)
    df = df[['corpus', 'token', 'tag', 'tag_STTS', 'lemma']]
    for a in algos[1:]:  # ignore baseline
        try:
            if not a == 'gpt3':  # not whole corpus lemmatized
                df[a] = pd.read_csv(f"lemmata/{corpus}/{a}-{corpus}.csv", encoding="utf-8", encoding_errors='ignore')["lemma_pred"]
        except Exception as e:  # corpus not lemmatized by algorithm
            print(e, f"{corpus} not lemmatized by {a}\n")
    data.append(df)
    #df.to_csv(f"lemmata/overall/{corpus}.csv", encoding="utf-8")
    df_all = df_all.append(df)
df_all['gpt3'] = gpt3_all['gpt3']

[Errno 2] No such file or directory: 'lemmata/empirist-web/stanza-pretagged-empirist-web.csv' empirist-web not lemmatized by stanza-pretagged

[Errno 2] No such file or directory: 'lemmata/empirist-web/stanza-empirist-web.csv' empirist-web not lemmatized by stanza

[Errno 2] No such file or directory: 'lemmata/empirist-web-norm/stanza-pretagged-empirist-web-norm.csv' empirist-web-norm not lemmatized by stanza-pretagged

[Errno 2] No such file or directory: 'lemmata/empirist-web-norm/stanza-empirist-web-norm.csv' empirist-web-norm not lemmatized by stanza

[Errno 2] No such file or directory: 'lemmata/germanc/rnntagger-germanc.csv' germanc not lemmatized by rnntagger

[Errno 2] No such file or directory: 'lemmata/ud-hdt/rnntagger-ud-hdt.csv' ud-hdt not lemmatized by rnntagger



In [89]:
# exclude GermanC, corpus not publicly available
df_all_wo_germanc = df_all[(df_all.corpus != 'germanc') & (df_all.corpus != 'germanc-norm')]

In [90]:
df_all_wo_germanc

,corpus,token,tag,tag_STTS,lemma,germalemma,gpt3,hanta-pretagged,hanta,rnntagger,simplemma,smorlemma,spacy2,spacy3,spacy33+,stanza-pretagged,stanza,trankit,treetagger
0,empirist-cmc-blog,…,PUNCT,$(,…,NaN,…,…,…,…,…,…,…,…,…,...,...,…,...
1,empirist-cmc-blog,das,PRON,PDS,der,NaN,der,der,der,der,der,NaN,der,der,der,der,der,der,die
2,empirist-cmc-blog,hört,VERB,VVFIN,hören,hören,hören,hören,hören,hören,hören,hören,hören,hören,hören,hören,hören,hören,hören
3,empirist-cmc-blog,sich,PRON,PRF,sich,NaN,sich,sich,sich,sich,er|es|sie,NaN,sich,sich,sich,er|es|sie,er|es|sie,er|es|sie,sich
4,empirist-cmc-blog,ja,UNK,PTKMA,ja,NaN,ja,ja,ja,ja,ja,ja,ja,ja,ja,ja,ja,ja,ja
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21299,ud-pud,Freund,NOUN,NN,Freund,Freund,,Freund,Freund,Freund,Freund,Freund,Freund,Freund,Freund,Freund,Freund,Freund,Freund
21300,ud-pud,des,DET,DT,der,NaN,,der,der,der,der,NaN,der,der,der,der,der,der,die
21301,ud-pud,Friedens,NOUN,NN,Frieden,Frieden,,Frieden,Frieden,Frieden,Frieden,Frieden,Frieden,Frieden,Frieden,Frieden,Frieden,Frieden,Frieden
21302,ud-pud,erklärte,VERB,VBC,erklären,erklären,,erklären,erklären,erklären,erklären,erklären,erklären,erklären,erklären,erklären,erklären,erklären,erklären


In [91]:
df_all_wo_germanc.to_csv("lemmata/overview/all.csv", encoding="utf-8")